# Inroduction

**Credit card complaints** are a vital indicator of consumer dissatisfaction and systemic issues in financial services. This case study aims to uncover patterns, trends, and insights from a dataset of consumer complaints related to credit cards. By analyzing this data, we can identify top issues, evaluate company responsiveness, and understand how complaints vary across time, geography, and submission channels. Understand patterns and drivers of consumer credit-card complaints and deliver a dashboard + analysis answering the business questions:

1. Trends over time
2. Top issue types.
3. Company responses.
4. Total complaints.
5. Timely response performance.
6. Cases In-Progress.
7. Submitted via.
8. State-wise complaints.
9. Daily complaints across months.

**Tools Used:**

1. **Microsoft Excel:** Used for initial data inspection, reviewing column headers, and understanding data types at a high level.
2. **Tableau:** For creating interactive dashboards and visualizations to share the findings.

# Prepare

**Key columns used:**

company, company_public_response, company_response_to_consumer, complaint_category, complaint_id, consumer_complaint_narrative, consumer_consent_provided, consumer_disputed, date_received, date_sent_to_company, dimension, issue, product, state, sub_issue, sub_product, submitted_via, tags, timely_response, zip_code, avg_no_of_days, number_of_records

**Checks to run immediately:**

* Ensure complaint_id unique.
* Parse date_received to Date type.
* Standardize categorical values (company names, submitted_via channels, state codes).
* Remove duplicates, trim whitespace, set consistent casing.
* Convert timely_response to logical/boolean (Yes/No or TRUE/FALSE).
* Handle missing date_sent_to_company (treat as NA) and company_response_to_consumer.

# Process

Data cleaning, transformation, and preparation for analysis.

In [ ]:
# Load packages

library(tidyverse)
library(lubridate)
library(readr)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     


── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [ ]:
# Import data

ccd <- read.csv("content/credit-card-complaints/Credit Card Data.csv")

In [ ]:
# Data structure

glimpse(ccd)

summary(ccd)

str(ccd)

Rows: 86,893
Columns: 25
$ company                      <chr> "Synchrony Financial", "JPMorgan Chase & …
$ company_public_response      <chr> "", "", "", "", "", "", "", "", "", "", "…
$ company_response_to_consumer <chr> "Closed with non-monetary relief", "Close…
$ complaint_category           <chr> "Billing disputes", "Closing/Cancelling a…
$ complaint_id                 <int> 1118322, 1111301, 1118893, 1097558, 11063…
$ consumer_complaint_narrative <chr> "", "", "", "", "", "", "", "", "", "", "…
$ consumer_consent_provided    <chr> "N/A", "N/A", "N/A", "N/A", "N/A", "N/A",…
$ consumer_consent_provided.1  <chr> " Other Includes Null", " Other Includes …
$ consumer_disputed            <chr> "No", "No", "Yes", "Yes", "No", "No", "No…
$ date_received                <chr> "2018-11-15", "2018-11-11", "2018-11-16",…
$ date_sent_to_company         <chr> "2018-11-15", "2018-11-11", "2018-11-16",…
$ dimension                    <chr> "Billing disputes", "Closing/Cancelling a…
$ issue        

   company          company_public_response company_response_to_consumer
 Length:86893       Length:86893            Length:86893                
 Class :character   Class :character        Class :character            
 Mode  :character   Mode  :character        Mode  :character            
                                                                        
                                                                        
                                                                        
                                                                        
 complaint_category  complaint_id     consumer_complaint_narrative
 Length:86893       Min.   :      1   Length:86893                
 Class :character   1st Qu.: 407591   Class :character            
 Mode  :character   Median :1183318   Mode  :character            
                    Mean   :1151672                               
                    3rd Qu.:1860506                               
              

'data.frame':	86893 obs. of  25 variables:
 $ company                     : chr  "Synchrony Financial" "JPMorgan Chase & Co." "Sterling Jewelers Inc." "Citibank" ...
 $ company_public_response     : chr  "" "" "" "" ...
 $ company_response_to_consumer: chr  "Closed with non-monetary relief" "Closed with explanation" "Closed with explanation" "Closed with explanation" ...
 $ complaint_category          : chr  "Billing disputes" "Closing/Cancelling account" "Bankruptcy" "Billing disputes" ...
 $ complaint_id                : int  1118322 1111301 1118893 1097558 1106302 1118818 1111396 1118698 1118872 1119008 ...
 $ consumer_complaint_narrative: chr  "" "" "" "" ...
 $ consumer_consent_provided   : chr  "N/A" "N/A" "N/A" "N/A" ...
 $ consumer_consent_provided.1 : chr  " Other Includes Null" " Other Includes Null" " Other Includes Null" " Other Includes Null" ...
 $ consumer_disputed           : chr  "No" "No" "Yes" "Yes" ...
 $ date_received               : chr  "2018-11-15" "2018-11-11" 

In [ ]:
# Clean data & standardize

ccd %>%
  mutate(
    complaint_id = as.character(complaint_id),
    date_received = as_date(date_received),
    date_sent_to_company = as_date(date_sent_to_company),
    state = str_to_upper(str_trim(state)),
    submitted_via = case_when(
      str_detect(submitted_via, regex("web|website|online", ignore_case=TRUE)) ~ "Web",
      str_detect(submitted_via, regex("referral", ignore_case=TRUE)) ~ "Referral",
      str_detect(submitted_via, regex("phone|call", ignore_case=TRUE)) ~ "Phone",
      str_detect(submitted_via, regex("postal|mail", ignore_case=TRUE)) ~ "Postal mail",
      str_detect(submitted_via, regex("fax", ignore_case=TRUE)) ~ "Fax",
      str_detect(submitted_via, regex("email", ignore_case=TRUE)) ~ "Email",
      TRUE ~ coalesce(submitted_via, "Other")
    ),
    timely_flag = case_when(
      tolower(timely_response) %in% c("yes","y","true","1") ~ 1,
      tolower(timely_response) %in% c("no","n","false","0") ~ 0,
      TRUE ~ NA_real_
    ),
    days_to_company = as.numeric(date_sent_to_company - date_received),
    year = year(date_received),
    month = floor_date(date_received, unit = "month"),
    week = floor_date(date_received, unit = "week"),
    day = as_date(date_received)
  )

company,company_public_response,company_response_to_consumer,complaint_category,complaint_id,consumer_complaint_narrative,consumer_consent_provided,consumer_consent_provided.1,consumer_disputed,date_received,⋯,zip_code,avg_no_of_days,donut,number_of_records,timely_flag,days_to_company,year,month,week,day
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,⋯,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<date>,<date>,<date>
Synchrony Financial,,Closed with non-monetary relief,Billing disputes,1118322,,N/A,Other Includes Null,No,2018-11-15,⋯,30024,0,0,1,1,0,2018,2018-11-01,2018-11-11,2018-11-15
JPMorgan Chase & Co.,,Closed with explanation,Closing/Cancelling account,1111301,,N/A,Other Includes Null,No,2018-11-11,⋯,6606,0,0,1,1,0,2018,2018-11-01,2018-11-11,2018-11-11
Sterling Jewelers Inc.,,Closed with explanation,Bankruptcy,1118893,,N/A,Other Includes Null,Yes,2018-11-16,⋯,54935,0,0,1,1,0,2018,2018-11-01,2018-11-11,2018-11-16
Citibank,,Closed with explanation,Billing disputes,1097558,,N/A,Other Includes Null,Yes,2018-11-02,⋯,5150,0,0,1,1,0,2018,2018-11-01,2018-10-28,2018-11-02
Discover,,Closed with explanation,Billing disputes,1106302,,N/A,Other Includes Null,No,2018-11-06,⋯,60646,0,0,1,1,0,2018,2018-11-01,2018-11-04,2018-11-06
TD Bank US Holding Company,,Closed with monetary relief,Delinquent account,1118818,,N/A,Other Includes Null,No,2018-11-16,⋯,92024,0,0,1,1,0,2018,2018-11-01,2018-11-11,2018-11-16
Bank of America,,Closed with non-monetary relief,Advertising and marketing,1111396,,N/A,Other Includes Null,No,2018-11-11,⋯,23112,0,0,1,1,0,2018,2018-11-01,2018-11-11,2018-11-11
JPMorgan Chase & Co.,,Closed with explanation,Delinquent account,1118698,,N/A,Other Includes Null,Yes,2018-11-16,⋯,75007,2,0,1,1,2,2018,2018-11-01,2018-11-11,2018-11-16
HSBC North America Holdings Inc.,,Closed with explanation,Billing disputes,1118872,,N/A,Other Includes Null,No,2018-11-16,⋯,20036,4,0,1,1,4,2018,2018-11-01,2018-11-11,2018-11-16


# Analyze


**Addressing each problem statement**

### Total complaints

In [ ]:
nrow(ccd)

[1] 86893

### Top issue

In [ ]:
ccd %>%
  count(issue, sort = TRUE) %>% slice_head(n=10)

issue,n
<chr>,<int>
Billing disputes,14688
Other,9049
Identity theft / Fraud / Embezzlement,8244
Closing/Cancelling account,6230
APR or interest rate,5426
Late fee,3550
Customer service / Customer relations,3424
Delinquent account,3135
Credit determination,2976


### Company response distribution

In [ ]:
ccd %>% count(company_public_response, sort=TRUE)

company_public_response,n
<chr>,<int>
,67148
Company has responded to the consumer and the CFPB and chooses not to provide a public response,12343
Company chooses not to provide a public response,6313
Company believes it acted appropriately as authorized by contract or law,837
Company believes complaint is the result of an isolated error,123
Company believes complaint represents an opportunity for improvement to better serve consumers,44
Company believes the complaint is the result of a misunderstanding,32
Company can't verify or dispute the facts in the complaint,20
Company disputes the facts presented in the complaint,17


### In progress

In [ ]:
ccd %>%
  filter(str_detect(tolower(company_response_to_consumer), "in progress|pending|open")) %>%
  count()

n
<int>
329


### Submitted via

In [ ]:
submitted_levels <- c("Web", "Email", "Phone", "Fax", "Postal Mail")

ccd %>%
  mutate(submitted_via = fct_expand(submitted_via, submitted_levels)) %>%
  count(submitted_via, name = "complaints") %>%
  mutate(share = complaints / sum(complaints)) %>%
  arrange(desc(complaints))

submitted_via,complaints,share
<fct>,<int>,<dbl>
Web,59889,6.892270e-01
Referral,14406,1.657901e-01
Phone,6502,7.482766e-02
Postal mail,5182,5.963656e-02
Fax,871,1.002382e-02
Email,42,4.833531e-04
,1,1.150841e-05


### State wise complaints

In [ ]:
state_counts <- ccd %>% count(state, sort = TRUE)
head(state_counts, 10)

,state,n
,<chr>,<int>
1,CA,12102
2,NY,8175
3,FL,7438
4,TX,5625
5,NJ,3821
6,PA,3385
7,IL,3240
8,OH,2990
9,VA,2940


### Daily complaints over months

In [ ]:
ccd %>%
  mutate(date_received = as.Date(date_received)) %>%   # convert to Date
  group_by(month = floor_date(date_received, "month"),
           day = day(date_received)) %>%               # extract day of month
  summarise(n = n(), .groups = "drop")

month,day,n
<date>,<int>,<int>
2015-12-01,1,85
2015-12-01,2,109
2015-12-01,3,20
2015-12-01,4,14
2015-12-01,5,139
2015-12-01,6,115
2015-12-01,7,65
2015-12-01,8,44
2015-12-01,9,28


# Conclusion

This analysis of credit card complaints reveals critical insights into consumer pain points and company responsiveness. Key takeaways include:


1. Complaints are rising steadily, especially during financial stress periods.
2. Billing disputes and unauthorized charges dominate complaint categories.
3. Timely responses are generally high, but some companies lag behind.
4. Web remains the most popular complaint submission channel.
5. Certain states show disproportionately high complaint volumes.


By leveraging Excel for exploration and Tableau for visualization, this case study provides a robust framework for understanding and acting on consumer feedback in the credit card industry.